<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/scraping_from_wikimedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=109623c7ac24cb61ee86c648c9b2791fa88e405bb09714c01da4a3a5dcef55c2
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import shutil
from fake_useragent import UserAgent
import pathlib
import time

In [3]:
def download_images(url, file_path):
  ua = UserAgent()
  header = {'user-agent':ua.chrome}
  r = requests.get(url, stream=True, headers=header)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)
  else: print(f'error {r.status_code}')

In [4]:
# collect instance urls
first_loop=True
move_to_next_page=True
instance_urls=[]

while move_to_next_page:
  # requests to wikimedia regarding category which I want to exploit images
  wikimedia_url = 'https://commons.wikimedia.org'
  category='Dog_breeds_by_name'
  category_url=f'/wiki/Category:{category}'
  
  if first_loop: 
    res = requests.get(wikimedia_url+category_url)
    time.sleep(1)
    first_loop=False
  else: res = requests.get(wikimedia_url+next_page_url)

  soup = BeautifulSoup(res.text, "html.parser")
  try:  next_page_url=soup.find_all(text="next page")[0].parent.attrs['href']
  except: move_to_next_page=False

  elems=soup.find_all(class_="CategoryTreeItem")

  for elem in elems:
    instance_url=elem.find('a').attrs['href']
    instance_urls.append(instance_url)

In [ ]:
# len(instance_urls)
# wikimedia_url+category_url
instance_urls

In [44]:
# collect image urls of each instance
for instance_url in instance_urls[:1]:
  res = requests.get(wikimedia_url+instance_url)
  time.sleep(1)
  soup = BeautifulSoup(res.text, "html.parser")

  extiw_class=soup.select('a[href^="https://www.wikidata.org/wiki/Q"]')
  try:  wikidata_id=pathlib.Path(extiw_class[0].attrs['href']).stem
  except:  continue
  path = "/content/imgs/" + wikidata_id
  os.makedirs(path,exist_ok=True)

  img_urls=[]
  move_to_next_page=True

  while move_to_next_page:
    res = requests.get(wikimedia_url+instance_url)
    time.sleep(1)
    soup = BeautifulSoup(res.text, "html.parser")

    try:  instance_url=soup.find_all(text="next page")[0].parent.attrs['href']
    except: move_to_next_page=False
    
    image_classes=soup.find_all(class_="galleryfilename galleryfilename-truncate")

    for image_class in image_classes:
      img_page_url=image_class.attrs['href']
      res = requests.get(wikimedia_url+img_page_url)
      soup = BeautifulSoup(res.text, "html.parser")
      img_url = soup.find(class_="fullImageLink").a.attrs['href']
      img_urls.append(img_url)

  for index, url in enumerate(img_urls):
      filename = 'image_' + str(index) + '.jpg'
      image_path = os.path.join(path, filename)
      print(url)
      print(image_path)
      download_images(url=url, file_path=image_path)
  print()

https://upload.wikimedia.org/wikipedia/commons/8/85/2013_Westminster_Kennel_Club_Dog_Show-_Affenpinscher_Joey_aka_GCH_Banana_Joe_V_Tani_Kazari_%288471258772%29.jpg
https://upload.wikimedia.org/wikipedia/commons/d/d8/Affenpinscher_%28Monkey_Terrier%29_from_1915.JPG
https://upload.wikimedia.org/wikipedia/commons/d/d8/Affenpinscher_circa_1915.jpg
https://upload.wikimedia.org/wikipedia/commons/2/25/Affenpinscher_dog.jpg
https://upload.wikimedia.org/wikipedia/commons/9/94/Affenpinscher_Molly.jpg
https://upload.wikimedia.org/wikipedia/commons/1/17/Affenpinscher.jpg
https://upload.wikimedia.org/wikipedia/commons/8/83/An_Affenpinscher.jpg
https://upload.wikimedia.org/wikipedia/commons/b/b2/Borismindre.jpg
https://upload.wikimedia.org/wikipedia/commons/6/6b/DocFile_Affen.jpg
https://upload.wikimedia.org/wikipedia/commons/9/96/Een_Affenpinscher%2C_Bestanddeelnr_190-0807.jpg
https://upload.wikimedia.org/wikipedia/commons/7/70/English_Toy_Terrier_and_Affenpinscher_by_Frances_C._Fairman.jpg
https:/

In [18]:
ls

image_0.jpg   image_12.jpg  image_1.jpg  image_4.jpg  image_7.jpg
image_10.jpg  image_13.jpg  image_2.jpg  image_5.jpg  image_8.jpg
image_11.jpg  image_14.jpg  image_3.jpg  image_6.jpg  image_9.jpg


In [15]:
%cd imgs/Q7254 

/content/imgs/Q7254


In [23]:
!ls -lh

total 100K
-rw-r--r-- 1 root root  15K Aug 25 07:23 image_0.jpg
-rw-r--r-- 1 root root 3.8K Aug 25 07:23 image_10.jpg
-rw-r--r-- 1 root root 5.2K Aug 25 07:23 image_11.jpg
-rw-r--r-- 1 root root 3.7K Aug 25 07:23 image_12.jpg
-rw-r--r-- 1 root root 6.8K Aug 25 07:23 image_13.jpg
-rw-r--r-- 1 root root 4.0K Aug 25 07:23 image_14.jpg
-rw-r--r-- 1 root root 4.2K Aug 25 07:23 image_1.jpg
-rw-r--r-- 1 root root 3.1K Aug 25 07:23 image_2.jpg
-rw-r--r-- 1 root root 3.3K Aug 25 07:23 image_3.jpg
-rw-r--r-- 1 root root 3.5K Aug 25 07:23 image_4.jpg
-rw-r--r-- 1 root root 5.2K Aug 25 07:23 image_5.jpg
-rw-r--r-- 1 root root 4.2K Aug 25 07:23 image_6.jpg
-rw-r--r-- 1 root root 3.1K Aug 25 07:23 image_7.jpg
-rw-r--r-- 1 root root 5.4K Aug 25 07:23 image_8.jpg
-rw-r--r-- 1 root root 5.2K Aug 25 07:23 image_9.jpg
